# Simulation 00 -- Synthetic Data Generator

## Purpose
Generate **5,000 synthetic cell tower sites** with **known ground truth FMV** that we can later compare against model predictions. This is the foundation for all stress tests in simulations 01-07.

## What we produce
- Realistic site features (property, location, demographics, network, zoning)
- A **known FMV function** so we have ground truth
- **Distorted current rents** that simulate 25 years of contractual escalation
- Output: `data/synthetic/synthetic_5k_sites.parquet`

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
N_SITES = 5000

print(f'Generating {N_SITES} synthetic cell tower sites...')

## Step 0.1 -- Generate Site Locations
Distribute sites across US-like geography with realistic metro clustering.

In [ ]:
# -----------------------------------------------------------------
# Define ~30 metro areas with realistic lat/lon and density weights
# -----------------------------------------------------------------
metros = [
    # (name, lat, lon, weight, pop_density_base, med_income_base, med_home_base)
    ('New York',      40.71, -74.01, 0.08, 10000, 75000, 550000),
    ('Los Angeles',   34.05, -118.24, 0.07, 7500, 68000, 650000),
    ('Chicago',       41.88, -87.63, 0.06, 6000, 62000, 280000),
    ('Houston',       29.76, -95.37, 0.05, 3500, 58000, 210000),
    ('Phoenix',       33.45, -112.07, 0.05, 3000, 55000, 310000),
    ('Philadelphia',  39.95, -75.17, 0.04, 5500, 60000, 250000),
    ('San Antonio',   29.42, -98.49, 0.03, 3000, 50000, 185000),
    ('San Diego',     32.72, -117.16, 0.03, 4000, 72000, 600000),
    ('Dallas',        32.78, -96.80, 0.05, 3800, 60000, 250000),
    ('Atlanta',       33.75, -84.39, 0.04, 3500, 58000, 260000),
    ('Miami',         25.76, -80.19, 0.04, 6000, 52000, 350000),
    ('Denver',        39.74, -104.99, 0.03, 3500, 68000, 450000),
    ('Seattle',       47.61, -122.33, 0.03, 4000, 80000, 550000),
    ('Boston',        42.36, -71.06, 0.03, 6500, 78000, 480000),
    ('Nashville',     36.16, -86.78, 0.03, 2500, 55000, 280000),
    ('Portland',      45.52, -122.68, 0.02, 3000, 65000, 420000),
    ('Las Vegas',     36.17, -115.14, 0.02, 2500, 52000, 300000),
    ('Minneapolis',   44.98, -93.27, 0.02, 3500, 68000, 280000),
    ('Charlotte',     35.23, -80.84, 0.02, 2800, 55000, 250000),
    ('Austin',        30.27, -97.74, 0.03, 3200, 65000, 380000),
    # Rural / sparse areas
    ('Rural Montana',  46.87, -110.36, 0.01, 50,  42000, 200000),
    ('Rural Wyoming',  42.75, -107.21, 0.01, 30,  55000, 225000),
    ('Rural Kansas',   38.50, -98.77, 0.01, 40,  48000, 130000),
    ('Rural Alabama',  32.80, -86.80, 0.01, 60,  38000, 120000),
    ('Rural Nebraska', 41.50, -99.80, 0.01, 25,  50000, 140000),
    ('Rural Oregon',   43.80, -120.50, 0.01, 35,  46000, 250000),
    ('Rural Maine',    45.25, -69.25, 0.01, 40,  50000, 180000),
    ('Rural Midwest',  40.00, -89.00, 0.02, 80,  52000, 160000),
    ('Rural South',    34.00, -82.00, 0.02, 70,  44000, 150000),
    ('Rural West',     37.00, -109.00, 0.02, 45,  48000, 200000),
]

metro_names   = [m[0] for m in metros]
metro_lats    = np.array([m[1] for m in metros])
metro_lons    = np.array([m[2] for m in metros])
metro_weights = np.array([m[3] for m in metros])
metro_weights = metro_weights / metro_weights.sum()  # normalise
metro_pop_density = np.array([m[4] for m in metros])
metro_med_income  = np.array([m[5] for m in metros])
metro_med_home    = np.array([m[6] for m in metros])

# Assign each site to a metro
metro_idx = np.random.choice(len(metros), size=N_SITES, p=metro_weights)

# Scatter within metro (urban sites tight, rural sites spread)
spread = np.where(metro_pop_density[metro_idx] > 1000, 0.15, 0.6)
lat = metro_lats[metro_idx] + np.random.normal(0, spread)
lon = metro_lons[metro_idx] + np.random.normal(0, spread * 1.3)

# Clamp to US bounds
lat = np.clip(lat, 25, 49)
lon = np.clip(lon, -125, -66)

# Assign region and state-proxy
metro_region = [metro_names[i] for i in metro_idx]
state_map = {m[0]: m[0].split()[-1][:2].upper() for m in metros}
state_map.update({'New York': 'NY', 'Los Angeles': 'CA', 'Chicago': 'IL',
                  'Houston': 'TX', 'Phoenix': 'AZ', 'Philadelphia': 'PA',
                  'San Antonio': 'TX', 'San Diego': 'CA', 'Dallas': 'TX',
                  'Atlanta': 'GA', 'Miami': 'FL', 'Denver': 'CO',
                  'Seattle': 'WA', 'Boston': 'MA', 'Nashville': 'TN',
                  'Portland': 'OR', 'Las Vegas': 'NV', 'Minneapolis': 'MN',
                  'Charlotte': 'NC', 'Austin': 'TX',
                  'Rural Montana': 'MT', 'Rural Wyoming': 'WY',
                  'Rural Kansas': 'KS', 'Rural Alabama': 'AL',
                  'Rural Nebraska': 'NE', 'Rural Oregon': 'OR',
                  'Rural Maine': 'ME', 'Rural Midwest': 'IL',
                  'Rural South': 'SC', 'Rural West': 'NM'})
state = [state_map[metro_names[i]] for i in metro_idx]

print(f'Sites distributed across {len(metros)} regions')
print(pd.Series(metro_region).value_counts().head(10))

## Step 0.2 -- Generate Property Characteristics

In [ ]:
# Property type (7 canonical types)
prop_types = ['commercial', 'rooftop', 'vacant_land', 'residential',
              'government', 'agricultural', 'industrial']
prop_weights = [0.30, 0.25, 0.20, 0.12, 0.05, 0.04, 0.04]
property_type = np.random.choice(prop_types, size=N_SITES, p=prop_weights)

# Structure type (conditional on property type)
structure_map = {
    'commercial':   ['monopole', 'lattice', 'concealed_stealth'],
    'rooftop':      ['rooftop'],
    'vacant_land':  ['monopole', 'lattice', 'guyed'],
    'residential':  ['monopole', 'concealed_stealth'],
    'government':   ['monopole', 'lattice'],
    'agricultural': ['lattice', 'guyed'],
    'industrial':   ['monopole', 'lattice'],
}
structure_type = [np.random.choice(structure_map[pt]) for pt in property_type]

# Tower height (conditional on structure type)
height_params = {
    'monopole': (120, 30), 'lattice': (180, 40), 'guyed': (250, 60),
    'rooftop': (0, 0), 'concealed_stealth': (80, 20),
}
tower_height_ft = np.array([
    max(0, np.random.normal(*height_params[st])) for st in structure_type
])

# Building height (rooftop & commercial only)
building_height_ft = np.where(
    np.isin(property_type, ['rooftop', 'commercial']),
    np.random.lognormal(3.5, 0.6, N_SITES),  # ~33 ft median
    0
)

# Lease area (sqft)
lease_area_sqft = np.random.lognormal(7.5, 0.8, N_SITES)  # median ~1800 sqft

# Parcel acreage (vacant land & agricultural mainly)
parcel_acreage = np.where(
    np.isin(property_type, ['vacant_land', 'agricultural']),
    np.random.lognormal(1.5, 1.0, N_SITES),  # median ~4.5 acres
    lease_area_sqft / 43560  # convert footprint to acres for others
)

# Type-conditional flags
has_building    = np.isin(property_type, ['rooftop', 'commercial', 'industrial']).astype(int)
is_ground_tower = np.isin(property_type, ['vacant_land', 'agricultural', 'residential']).astype(int)
has_parcel      = np.isin(property_type, ['vacant_land', 'agricultural', 'residential']).astype(int)

# Construction year
construction_year = np.random.randint(1985, 2024, N_SITES)
construction_age  = 2026 - construction_year

# Site access type (ordinal: 1=unimproved, 2=gravel, 3=paved, 4=elevator)
access_probs = {  # conditional on property type
    'rooftop':      [0.0, 0.0, 0.3, 0.7],
    'commercial':   [0.05, 0.15, 0.7, 0.1],
    'vacant_land':  [0.3, 0.4, 0.3, 0.0],
    'residential':  [0.1, 0.3, 0.6, 0.0],
    'government':   [0.05, 0.15, 0.7, 0.1],
    'agricultural': [0.4, 0.4, 0.2, 0.0],
    'industrial':   [0.1, 0.2, 0.6, 0.1],
}
site_access_ordinal = np.array([
    np.random.choice([1, 2, 3, 4], p=access_probs[pt]) for pt in property_type
])

print('Property type distribution:')
print(pd.Series(property_type).value_counts())

## Step 0.3 -- Generate Market & Demographic Features
Grounded in metro-level baselines with realistic per-site variation.

In [ ]:
# Census Population Density (persons / sq km)
base_pop_density = metro_pop_density[metro_idx].astype(float)
census_population_density = np.maximum(
    5, base_pop_density * np.random.lognormal(0, 0.5, N_SITES)
)

# Population within 3 km radius
census_population_3km = (census_population_density * np.pi * 9 * 
                          np.random.uniform(0.6, 1.4, N_SITES)).astype(int)

# Median Income
base_income = metro_med_income[metro_idx].astype(float)
census_median_income = np.maximum(
    20000, base_income * np.random.lognormal(0, 0.25, N_SITES)
)

# Median Home Value (strong FMV driver)
base_home = metro_med_home[metro_idx].astype(float)
census_median_home_value = np.maximum(
    50000, base_home * np.random.lognormal(0, 0.3, N_SITES)
)

# Census Median Rent (residential rent anchor)
census_median_rent = census_median_home_value * np.random.uniform(0.003, 0.006, N_SITES)

# Vacancy rate
census_vacancy_rate = np.clip(np.random.beta(2, 15, N_SITES), 0.01, 0.4)

# Urban/Rural class (1=rural, 2=small_town, 3=suburban, 4=urban_core)
urban_rural_bins = np.digitize(census_population_density, [100, 1000, 5000]) + 1
urban_rural_class = np.clip(urban_rural_bins, 1, 4)

# Underlying land value ($/sqft)
base_land = np.where(urban_rural_class >= 3, 
                      np.random.lognormal(3.0, 0.8, N_SITES),  # ~$20/sqft
                      np.random.lognormal(1.0, 0.8, N_SITES))  # ~$2.7/sqft
underlying_land_value_psf = np.maximum(0.1, base_land)

# HUD Small Area Fair Market Rent (universal coverage)
hud_safmr = census_median_rent * np.random.uniform(0.85, 1.15, N_SITES)

# Property tax rate
property_tax_rate = np.random.uniform(0.005, 0.03, N_SITES)

# Topography (ordinal: 1=flat, 2=hilly, 3=mountainous)
topography_class = np.random.choice([1, 2, 3], N_SITES, p=[0.5, 0.35, 0.15])
terrain_elevation_variance_m = topography_class * np.random.exponential(15, N_SITES)

# POI density and building density
poi_density_1km = (census_population_density / 50 * 
                    np.random.lognormal(0, 0.5, N_SITES)).astype(int)
building_density_500m = (census_population_density / 20 * 
                          np.random.lognormal(0, 0.4, N_SITES)).astype(int)

# Distance to highway (km)
distance_to_highway_km = np.where(
    urban_rural_class >= 3,
    np.random.exponential(2, N_SITES),
    np.random.exponential(10, N_SITES)
)

# Ground elevation (ft)
ground_elevation_ft = np.random.uniform(100, 8000, N_SITES)

# Unemployment rate
unemployment_rate_local = np.clip(np.random.normal(0.05, 0.02, N_SITES), 0.02, 0.15)

print('Key market features generated.')
print(f'  Population density range: {census_population_density.min():.0f} - {census_population_density.max():.0f}')
print(f'  Median home value range:  ${census_median_home_value.min():,.0f} - ${census_median_home_value.max():,.0f}')
print(f'  Land value ($/sqft) range: ${underlying_land_value_psf.min():.2f} - ${underlying_land_value_psf.max():.2f}')

## Step 0.4 -- Generate Zoning, Competition & Network Features

In [ ]:
# ---- Zoning / Regulatory (Domain 2) ----
zoning_types = ['residential', 'commercial', 'industrial', 'agricultural', 'mixed_use']
# Correlated with property type
zoning_map = {
    'commercial':   [0.1, 0.5, 0.1, 0.0, 0.3],
    'rooftop':      [0.2, 0.4, 0.1, 0.0, 0.3],
    'vacant_land':  [0.2, 0.1, 0.1, 0.4, 0.2],
    'residential':  [0.6, 0.1, 0.0, 0.1, 0.2],
    'government':   [0.1, 0.3, 0.2, 0.1, 0.3],
    'agricultural': [0.1, 0.0, 0.0, 0.8, 0.1],
    'industrial':   [0.0, 0.2, 0.6, 0.0, 0.2],
}
zoning_district_type = [
    np.random.choice(zoning_types, p=zoning_map[pt]) for pt in property_type
]

# Permit approval difficulty (1-5)
permit_approval_difficulty = np.clip(
    np.random.poisson(2, N_SITES) + (urban_rural_class - 1), 1, 5
)

# WRLURI regulatory index (higher = more restrictive)
wrluri_index = np.random.normal(0, 1, N_SITES)  # standardised
wrluri_index = np.where(urban_rural_class >= 3, wrluri_index + 0.5, wrluri_index - 0.3)

# Colocation required
colocation_required = (np.random.random(N_SITES) < 0.25).astype(int)

# Height restriction
height_restriction_ft = np.where(
    urban_rural_class >= 3,
    np.random.choice([100, 120, 150, 200], N_SITES),
    np.random.choice([150, 200, 300, 999], N_SITES)
)

# Has zoning data flag
has_zoning_data = (np.random.random(N_SITES) < 0.85).astype(int)

# ---- Competition / Supply (Domain 7) ----
# FCC tower count within 5 km (correlated with population density)
fcc_tower_count_5km = np.maximum(0, 
    (np.log(census_population_density + 1) * 3 +
     np.random.normal(0, 2, N_SITES)).astype(int)
)

fcc_tower_density_sqkm = fcc_tower_count_5km / (np.pi * 25)  # area of 5km circle

competitor_tower_distance_km = np.maximum(
    0.1, np.where(urban_rural_class >= 3, 
                   np.random.exponential(1.5, N_SITES),
                   np.random.exponential(8, N_SITES))
)

tower_per_capita = fcc_tower_count_5km / np.maximum(census_population_3km, 100) * 1000

# Scarcity index (0-1, lower = scarcer = more valuable)
scarcity_index = np.clip(
    1 - (1 / (fcc_tower_count_5km + 1)) * (topography_class / 3) *
    (permit_approval_difficulty / 5),
    0, 1
)

# Landlord concentration (distinct landlords in area)
landlord_concentration = np.maximum(1, 
    (fcc_tower_count_5km * 0.6 + np.random.poisson(2, N_SITES)).astype(int)
)

# Natural hazard score (0-1)
composite_hazard_score = np.random.beta(2, 8, N_SITES)

# ---- Network Parameters (Domain 4 -- Z_network) ----
tenant_count = np.random.poisson(1.5, N_SITES) + 1  # 1 to ~5
tenant_count = np.clip(tenant_count, 1, 5)

tech_gen_probs = [0.02, 0.05, 0.35, 0.45, 0.13]  # 2G, 3G, 4G, 5G_sub6, 5G_mm
primary_technology = np.random.choice([1, 2, 3, 4, 5], N_SITES, p=tech_gen_probs)

coverage_critical = (np.random.random(N_SITES) < np.where(
    urban_rural_class <= 2, 0.30, 0.08  # rural sites more likely sole-coverage
)).astype(int)

capacity_utilization_pct = np.clip(
    np.random.beta(3, 4, N_SITES) * 100, 5, 99
)

high_traffic = (capacity_utilization_pct > 70).astype(int)

backhaul_type = np.random.choice(
    [1, 2, 3, 4],  # 1=satellite, 2=copper, 3=microwave, 4=fiber
    N_SITES,
    p=[0.03, 0.07, 0.25, 0.65]
)

has_fiber_connection = (backhaul_type == 4).astype(int)

network_substitutability_score = np.clip(
    np.random.beta(4, 3, N_SITES) * (1 - coverage_critical * 0.5), 0, 1
)

# Search ring alternatives
search_ring_alternative_count = np.maximum(0,
    (fcc_tower_count_5km * 0.3 + np.random.poisson(1, N_SITES) - 
     coverage_critical * 2).astype(int)
)

inter_site_distance_km = np.maximum(0.2,
    np.where(urban_rural_class >= 3,
             np.random.exponential(1.2, N_SITES),
             np.random.exponential(5, N_SITES))
)

antenna_count = tenant_count * np.random.choice([3, 4, 6], N_SITES, p=[0.3, 0.5, 0.2])

structure_capacity_remaining = np.clip(
    np.random.beta(3, 2, N_SITES) - tenant_count * 0.15, 0, 1
)

# Site priority tier (1=critical, 2=important, 3=fungible)
site_priority_tier = np.where(
    coverage_critical == 1, 1,
    np.where(capacity_utilization_pct > 60, 2, 3)
)

print('Zoning, competition, and network features generated.')
print(f'  Tower count 5km range: {fcc_tower_count_5km.min()} - {fcc_tower_count_5km.max()}')
print(f'  Tenant count distribution: {np.bincount(tenant_count)[1:]}')
print(f'  Coverage critical sites: {coverage_critical.sum()} ({coverage_critical.mean()*100:.1f}%)')

## Step 0.5 -- Generate TRUE FMV (Ground Truth)

This is the **known function** the pipeline must recover. The coefficients are chosen to produce realistic rent ranges ($500-$5,000/mo).

```
log(true_FMV) = 4.5
  + 0.35 * log(census_median_home_value)
  + 0.20 * log(population_density + 1)
  + 0.15 * log(underlying_land_value + 1)
  + 0.10 * tenant_count
  + 0.25 * coverage_critical
  - 0.10 * log(fcc_tower_count_5km + 1)
  + 0.15 * scarcity_index
  + 0.08 * (permit_approval_difficulty - 3)
  + 0.05 * has_fiber_connection
  + epsilon  ~ N(0, 0.08)
```

In [ ]:
# Compute log(true_FMV)
log_true_fmv = (
    4.5
    + 0.35 * np.log(census_median_home_value)
    + 0.20 * np.log(census_population_density + 1)
    + 0.15 * np.log(underlying_land_value_psf + 1)
    + 0.10 * tenant_count
    + 0.25 * coverage_critical
    - 0.10 * np.log(fcc_tower_count_5km + 1)
    + 0.15 * scarcity_index
    + 0.08 * (permit_approval_difficulty - 3)
    + 0.05 * has_fiber_connection
    + np.random.normal(0, 0.08, N_SITES)  # irreducible noise
)

true_fmv = np.exp(log_true_fmv)

# Scale to realistic range ($300 - $6,000)
# The raw exponential may need rescaling; let's check and adjust
raw_median = np.median(true_fmv)
target_median = 1500  # target median monthly rent ~$1,500
scale_factor = target_median / raw_median
true_fmv = true_fmv * scale_factor

# Clip extreme values
true_fmv = np.clip(true_fmv, 200, 8000)

print(f'TRUE FMV distribution (ground truth):')
print(f'  Min: ${true_fmv.min():,.0f}/mo')
print(f'  Q25: ${np.percentile(true_fmv, 25):,.0f}/mo')
print(f'  Median: ${np.median(true_fmv):,.0f}/mo')
print(f'  Q75: ${np.percentile(true_fmv, 75):,.0f}/mo')
print(f'  Max: ${true_fmv.max():,.0f}/mo')
print(f'  Scale factor applied: {scale_factor:.4f}')

## Step 0.6 -- Generate Distorted Current Rents
Simulate 25 years of contractual escalation, origination noise, and lease amendments.

In [ ]:
# Lease age (years since origination)
lease_age_years = np.random.uniform(2, 25, N_SITES)

# Remaining term
total_term = np.random.choice([5, 10, 15, 20, 25], N_SITES, p=[0.1, 0.2, 0.3, 0.25, 0.15])
remaining_term_years = np.maximum(0, total_term - lease_age_years + np.random.uniform(0, 5, N_SITES))

# Escalation type
esc_types = ['fixed_3pct', 'fixed_2pct', 'cpi_linked', 'flat']
esc_weights = [0.40, 0.20, 0.25, 0.15]
escalation_type = np.random.choice(esc_types, N_SITES, p=esc_weights)

# Escalation rate
esc_rate_map = {
    'fixed_3pct': 0.03,
    'fixed_2pct': 0.02,
    'cpi_linked': 0.025,  # avg CPI over 25 yrs
    'flat': 0.0,
}
escalation_rate = np.array([esc_rate_map[e] for e in escalation_type])

# Origination noise: the base rent at lease inception deviated from true FMV
origination_noise_std = 0.10  # 10% noise
origination_factor = np.random.lognormal(0, origination_noise_std, N_SITES)

# True FMV at inception (simplified: assume FMV grows at ~2%/yr historically)
historical_growth_rate = 0.02
true_fmv_at_inception = true_fmv / (1 + historical_growth_rate) ** lease_age_years

# Base rent at inception = true FMV at inception * origination noise
base_rent_at_inception = true_fmv_at_inception * origination_factor

# Cumulative escalation factor
cumulative_escalation_factor = (1 + escalation_rate) ** lease_age_years

# Current rent = base rent * escalation
current_monthly_rent = base_rent_at_inception * cumulative_escalation_factor

# Add some random amendments (5% of sites had a rent reset)
amendment_mask = np.random.random(N_SITES) < 0.05
amendment_count = amendment_mask.astype(int)
current_monthly_rent[amendment_mask] = (
    true_fmv[amendment_mask] * np.random.lognormal(0, 0.05, amendment_mask.sum())
)

# De-escalated base rent (what the model will compute)
de_escalated_base_rent = current_monthly_rent / cumulative_escalation_factor

# Clip extreme rents
current_monthly_rent = np.clip(current_monthly_rent, 100, 15000)

# Compute true classification
true_deviation_pct = (current_monthly_rent - true_fmv) / true_fmv
true_class = np.where(true_deviation_pct > 0.15, 'OVERPAYING',
             np.where(true_deviation_pct < -0.15, 'UNDERPAYING', 'AT_MARKET'))

print(f'Current rent distribution (distorted):')
print(f'  Min: ${current_monthly_rent.min():,.0f}/mo')
print(f'  Median: ${np.median(current_monthly_rent):,.0f}/mo')
print(f'  Max: ${current_monthly_rent.max():,.0f}/mo')
print(f'\nTrue classification breakdown:')
for c in ['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']:
    n = (true_class == c).sum()
    print(f'  {c}: {n} ({n/N_SITES*100:.1f}%)')
print(f'\nDeviation stats: mean={true_deviation_pct.mean()*100:.1f}%, std={true_deviation_pct.std()*100:.1f}%')

## Step 0.7 -- Assemble DataFrame and Save

In [ ]:
df = pd.DataFrame({
    # Identity / Location
    'site_id':              [f'SIM_{i:05d}' for i in range(N_SITES)],
    'lat':                  lat,
    'lon':                  lon,
    'state':                state,
    'metro_region':         metro_region,
    
    # Property Characteristics (Domain 1 / Domain 3)
    'property_type':        property_type,
    'structure_type':       structure_type,
    'tower_height_ft':      np.round(tower_height_ft, 1),
    'building_height_ft':   np.round(building_height_ft, 1),
    'lease_area_sqft':      np.round(lease_area_sqft, 0),
    'parcel_acreage':       np.round(parcel_acreage, 2),
    'construction_year':    construction_year,
    'construction_age':     construction_age,
    'site_access_ordinal':  site_access_ordinal,
    'has_building':         has_building,
    'is_ground_tower':      is_ground_tower,
    'has_parcel':           has_parcel,
    
    # Zoning / Regulatory (Domain 2)
    'zoning_district_type':       zoning_district_type,
    'permit_approval_difficulty':  permit_approval_difficulty,
    'wrluri_index':               np.round(wrluri_index, 3),
    'colocation_required':        colocation_required,
    'height_restriction_ft':      height_restriction_ft,
    'has_zoning_data':            has_zoning_data,
    
    # Market / Demographics (Domain 5)
    'census_population_density':  np.round(census_population_density, 1),
    'census_population_3km':      census_population_3km,
    'census_median_income':       np.round(census_median_income, 0),
    'census_median_home_value':   np.round(census_median_home_value, 0),
    'census_median_rent':         np.round(census_median_rent, 0),
    'census_vacancy_rate':        np.round(census_vacancy_rate, 4),
    'urban_rural_class':          urban_rural_class,
    'underlying_land_value_psf':  np.round(underlying_land_value_psf, 2),
    'hud_safmr':                  np.round(hud_safmr, 0),
    'property_tax_rate':          np.round(property_tax_rate, 4),
    'topography_class':           topography_class,
    'terrain_elevation_variance_m': np.round(terrain_elevation_variance_m, 1),
    'poi_density_1km':            poi_density_1km,
    'building_density_500m':      building_density_500m,
    'distance_to_highway_km':     np.round(distance_to_highway_km, 2),
    'ground_elevation_ft':        np.round(ground_elevation_ft, 1),
    'unemployment_rate_local':    np.round(unemployment_rate_local, 4),
    
    # Competition / Supply (Domain 7)
    'fcc_tower_count_5km':          fcc_tower_count_5km,
    'fcc_tower_density_sqkm':       np.round(fcc_tower_density_sqkm, 4),
    'competitor_tower_distance_km':  np.round(competitor_tower_distance_km, 2),
    'tower_per_capita':             np.round(tower_per_capita, 4),
    'scarcity_index':               np.round(scarcity_index, 4),
    'landlord_concentration':       landlord_concentration,
    'composite_hazard_score':       np.round(composite_hazard_score, 4),
    
    # Network Parameters (Domain 4 -- Z_network)
    'tenant_count':                   tenant_count,
    'primary_technology':             primary_technology,
    'coverage_critical':              coverage_critical,
    'capacity_utilization_pct':       np.round(capacity_utilization_pct, 1),
    'high_traffic':                   high_traffic,
    'backhaul_type':                  backhaul_type,
    'has_fiber_connection':           has_fiber_connection,
    'network_substitutability_score': np.round(network_substitutability_score, 4),
    'search_ring_alternative_count':  search_ring_alternative_count,
    'inter_site_distance_km':         np.round(inter_site_distance_km, 2),
    'antenna_count':                  antenna_count,
    'structure_capacity_remaining':   np.round(structure_capacity_remaining, 4),
    'site_priority_tier':             site_priority_tier,
    
    # Lease / Contract (Control features)
    'lease_age_years':              np.round(lease_age_years, 2),
    'remaining_term_years':         np.round(remaining_term_years, 2),
    'escalation_type':              escalation_type,
    'escalation_rate':              escalation_rate,
    'cumulative_escalation_factor': np.round(cumulative_escalation_factor, 4),
    'amendment_count':              amendment_count,
    'de_escalated_base_rent':       np.round(de_escalated_base_rent, 2),
    
    # Target-adjacent (used for pseudo-labeling, NOT model input)
    'current_monthly_rent':  np.round(current_monthly_rent, 2),
    
    # GROUND TRUTH (hidden from model -- used for evaluation only)
    'true_fmv':              np.round(true_fmv, 2),
    'true_deviation_pct':    np.round(true_deviation_pct, 4),
    'true_class':            true_class,
})

print(f'DataFrame shape: {df.shape}')
print(f'Columns: {len(df.columns)}')
df.head(3)

In [ ]:
# Save to parquet
out_dir = os.path.join('..', 'data', 'synthetic')
os.makedirs(out_dir, exist_ok=True)

out_path = os.path.join(out_dir, 'synthetic_5k_sites.parquet')
df.to_parquet(out_path, index=False)
print(f'Saved to {out_path}')
print(f'File size: {os.path.getsize(out_path) / 1024:.0f} KB')

## Quick Sanity Plots

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. True FMV distribution
axes[0, 0].hist(df['true_fmv'], bins=60, color='steelblue', edgecolor='white', alpha=0.8)
axes[0, 0].set_title('True FMV Distribution')
axes[0, 0].set_xlabel('$/month')

# 2. Current rent vs True FMV scatter
colors = {'OVERPAYING': 'red', 'AT_MARKET': 'green', 'UNDERPAYING': 'blue'}
for cls in ['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']:
    mask = df['true_class'] == cls
    axes[0, 1].scatter(df.loc[mask, 'true_fmv'], df.loc[mask, 'current_monthly_rent'],
                       alpha=0.3, s=5, c=colors[cls], label=cls)
axes[0, 1].plot([0, 8000], [0, 8000], 'k--', lw=1, label='y=x')
axes[0, 1].set_xlabel('True FMV ($/mo)')
axes[0, 1].set_ylabel('Current Rent ($/mo)')
axes[0, 1].set_title('Current Rent vs True FMV')
axes[0, 1].legend(fontsize=8)

# 3. Deviation distribution
axes[0, 2].hist(df['true_deviation_pct'] * 100, bins=60, color='coral', edgecolor='white', alpha=0.8)
axes[0, 2].axvline(0, color='black', ls='--', lw=1)
axes[0, 2].set_title('Rent Deviation from True FMV (%)')
axes[0, 2].set_xlabel('Deviation %')

# 4. FMV by property type
df.boxplot(column='true_fmv', by='property_type', ax=axes[1, 0], vert=True)
axes[1, 0].set_title('True FMV by Property Type')
axes[1, 0].set_xlabel('')
plt.sca(axes[1, 0])
plt.xticks(rotation=45, ha='right', fontsize=8)

# 5. Lease age vs deviation
axes[1, 1].scatter(df['lease_age_years'], df['true_deviation_pct'] * 100,
                   alpha=0.15, s=5, c='purple')
axes[1, 1].axhline(0, color='black', ls='--', lw=1)
axes[1, 1].set_xlabel('Lease Age (years)')
axes[1, 1].set_ylabel('Deviation from True FMV (%)')
axes[1, 1].set_title('Lease Age vs Rent Deviation (H5 test)')

# 6. Geographic scatter
sc = axes[1, 2].scatter(df['lon'], df['lat'], c=df['true_fmv'], cmap='YlOrRd',
                        s=3, alpha=0.4, vmin=500, vmax=4000)
plt.colorbar(sc, ax=axes[1, 2], label='True FMV ($/mo)')
axes[1, 2].set_title('Geographic Distribution (colored by FMV)')
axes[1, 2].set_xlabel('Longitude')
axes[1, 2].set_ylabel('Latitude')

plt.suptitle('Simulation 00 -- Synthetic Data Sanity Check', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics
print('='*60)
print('SYNTHETIC DATA SUMMARY')
print('='*60)
print(f'Total sites:       {len(df)}')
print(f'Features:          {len(df.columns) - 3} (excl. ground truth cols)')
print(f'Property types:    {df["property_type"].nunique()}')
print(f'States:            {df["state"].nunique()}')
print(f'Metro regions:     {df["metro_region"].nunique()}')
print(f'\nTrue FMV:  median ${df["true_fmv"].median():,.0f}/mo, '
      f'range ${df["true_fmv"].min():,.0f}-${df["true_fmv"].max():,.0f}')
print(f'Current Rent: median ${df["current_monthly_rent"].median():,.0f}/mo, '
      f'range ${df["current_monthly_rent"].min():,.0f}-${df["current_monthly_rent"].max():,.0f}')
print(f'\nCorrelation (true_fmv, current_rent): {df["true_fmv"].corr(df["current_monthly_rent"]):.3f}')
print(f'\nReady for simulations 01-07.')